# Data Loading and Storage

In [ ]:
import numpy as np
import pandas as pd
import requests
import io
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize= (10,6))
np.set_printoptions(precision=4, suppress=True)

In [ ]:
url ='https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/ex1.csv'
df = pd.read_csv(url)
df

In [ ]:
url ='https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/ex1.csv'
df = pd.read_csv(url)
df

In [ ]:
url= 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/ex2.csv' # this file does not have a header row
df = pd.read_csv(url)
df

In [ ]:
df = pd.read_csv(url, header=None) # use this when there is no header
df

In [ ]:
df = pd.read_csv(url, names=['a', 'b', 'c', 'd', 'message']) # use this when there is no header row
df

In [ ]:
names=['a', 'b', 'c', 'd', 'message'] # defines the columns names
df = pd.read_csv(url, names=names, index_col='message') # sets the message column to be the index labels
df

In [ ]:
url = 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/csv_mindex.csv'

df = pd.read_csv(url, index_col=['key1', 'key2'])
df

In [ ]:
# this table doesn't have a fixed delimiter, saved as a txt file.
url = 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/ex3.txt'

# the sep is a regex for any one or more white space characters \s+
df = pd.read_csv(url, sep='\s+') # pandas infers that the first column is the index since it has no header
df

In [ ]:
url = 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/ex4.csv'

df = pd.read_csv(url, skiprows=[0,2,3]) # this skips the first, third, and fouth rows
df

In [ ]:
url = 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/ex5.csv'

df = pd.read_csv(url)
pd.isnull(df) # returns a boolean matrix indicating the null values


In [ ]:
# we can specify sentinels we want for missing vlaues for each column in a dic
sentinels = {'message': ['foo', 'NA'], 'something': ['two']} # sets the values we want assigned as NaN
url = 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/ex5.csv'

df = pd.read_csv(url, na_values=sentinels)
df


### Reading Text Files in Pieces

In [ ]:
url = 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/ex6.csv'

df = pd.read_csv(url, nrows=5) # only loads the first 5 rows
df

In [ ]:
# to split into chunck
url = 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/ex6.csv'

chuncker = pd.read_csv(url, chunksize=1000) # only loads the first 5 rows
chuncker # returns a TextFileReader object we can iterate over
tot = pd.Series()
for piece in chuncker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot= tot.sort_values(ascending=False)
tot[:10]

### Writing Data to Text Format

In [ ]:
url = 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/ex5.csv'

data = pd.read_csv(url)
data

In [ ]:
import sys
# this prints to the console with the '|' as the specified delimiter
data.to_csv(sys.stdout, sep='|')

In [ ]:
# missing vlaues appear as empty strings or spaces. We can use a sentinal value to fix this
data.to_csv(sys.stdout, na_rep='NULL')

In [ ]:
# be default, row and colimn labels are assigned. We can disable this
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

### Working with Delimited Formats

In [ ]:
url = 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/ex7.csv'

data = pd.read_csv(url)
data.to_csv('ex7.csv') # this write a .csv file to the current directory
import csv # this library helps us when there is malfrmed lines in a .csv
f = open('ex7.csv') # creates a TextIOWrapper
reader = csv.reader(f) # f is ther iterable object
# Iterating through the reader like a file yields tuples of values with any quote characters removed
for line in reader:
    print(line)


In [ ]:
# to wrangle this into the form we need we first read the files into a list of lines
with open('ex7.csv') as f:
    lines = list(csv.reader(f))
# then we split the lines into the header line and the data lines
header, values = lines[0], lines[1:] # the lines[1:] means we start at the second index and assign the rest to the values
# next we create a dictionary of data columsn to transpose rows to columns
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict


In [ ]:
# to define a new format with a different delimiter, string quoting, or line terminator, we define a simple subclass of csv.Dialect
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL
# here we use the subclass to apply the new format
with open('ex7.csv') as f:
    reader = csv.reader(f, dialect=my_dialect)
    for line in reader:
        print(line)


In [ ]:
# this time we use a '|' delimeter
with open('ex7.csv') as f:
    reader = csv.reader(f, delimiter='|')
    for line in reader:
        print(line)

In [ ]:
import os
os.remove('ex7.csv')

### JSON Data
the object types are dictionaries, arrays( list), strings, numbers, booleand, and nulls. All keys in an object must be strings.

In [ ]:
# example JSON
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""
import json
result = json.loads(obj) # coverts a JSON string to Python form.
result

In [ ]:
asjson = json.dumps(result) # converts a Python object back into JSON
asjson

In [ ]:
# we can pass a list of dicts (which were previousy JSON objects) to the DataFrame constructor and select a subset of the data fields
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

In [ ]:
url = 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/example.json'

data = pd.read_json(url)
data


In [ ]:
print(data.to_json()) # exports data from pandas to JSON
print(data.to_json(orient='records')) # creates a different orientation

### XML and HTML: Web Scraping
conda install lxml

In [ ]:
def download_file(url):
    
    data = pd.read_json(url)
    return data

download_file(url)



In [ ]:
url = 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/fdic_failed_bank_list.html'
tables = pd.read_html(url)
print(tables)
print(type(tables))
len(tables)

In [ ]:
failures = tables[0] # pulls out list
failures.head()

In [ ]:
# compute the number of bank failures by year
close_timestamps = pd.to_datetime(failures['Closing Date'])
print(close_timestamps.head())
close_timestamps.dt.year.value_counts()

### Parsing XML with lxml.objectify


In [ ]:
# to download a file from the web, use requests.get and write to the file format of chioce
url = 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/datasets/mta_perf/Performance_MNR.xml'
r = requests.get(url)
with open("MNR.xml", 'wb') as f:
    f.write(r.content)

In [276]:
# using lxml.objectify, we parse the file and get a referecne to the root node of the xml file with getroot
from lxml import objectify
path ='MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

os.remove('MNR.xml')

In [277]:
# the root.INDICATOR returns a generator yielding each <INDICATOR> XML element. For each record, we can populate a dict of tag names (like YTD_ACTUAL) to data values (excluding a few tags)
print(root.INDICATOR)

data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)
data

[{'AGENCY_NAME': 'Metro-North Railroad',
  'INDICATOR_NAME': 'On-Time Performance (West of Hudson)',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.\n',
  'PERIOD_YEAR': 2008,
  'PERIOD_MONTH': 1,
  'CATEGORY': 'Service Indicators',
  'FREQUENCY': 'M',
  'INDICATOR_UNIT': '%',
  'YTD_TARGET': 95.0,
  'YTD_ACTUAL': 96.9,
  'MONTHLY_TARGET': 95.0,
  'MONTHLY_ACTUAL': 96.9},
 {'AGENCY_NAME': 'Metro-North Railroad',
  'INDICATOR_NAME': 'On-Time Performance (West of Hudson)',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate 

In [ ]:
perf = pd.DataFrame(data) # convert this list of dicts into a DataFrame
perf.head()

In [ ]:
# XML can get much more complicated. Each tag can have metadata
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()
root

In [ ]:
root.get('href')

In [ ]:
root.text

### Binary Data Formats
best way to store data in a binary format is with Python's pickle serialization. Picke are recommended for a short term storage format. They are hard to keep stable over time with new format versions

In [ ]:
url ='https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/ex1.csv'
frame = pd.read_csv(url)
frame

In [278]:
frame.to_pickle('frame_pickle') # saves to a pickle file
pd.read_pickle('frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo
